In [1]:
import json

with open("../auth/nyc-open-data.json", "r") as f:
    nyc_auth = json.load(f)
    
import pysocrata
nyc_datasets = pysocrata.get_datasets(**nyc_auth)

nyc_datasets = [d for d in nyc_datasets if d['resource']['type'] != 'story']

nyc_types = [d['resource']['type'] for d in nyc_datasets]
volcab_map = {'dataset': 'table', 'href': 'link', 'map': 'geospatial dataset', 'file': 'blob'}
nyc_types = list(map(lambda d: volcab_map[d], nyc_types))

nyc_endpoints = [d['resource']['id'] for d in nyc_datasets]

In [2]:
import numpy as np
geospatial_indices = np.nonzero([t == 'geospatial dataset' for t in nyc_types])
geospatial_endpoints = np.array(nyc_endpoints)[geospatial_indices]
geospatial_datasets = np.array(nyc_datasets)[geospatial_indices]

In [4]:
geospatial_endpoints

array(['3qz8-muuu', 'uxpt-rzip', 'u6su-4fpt', 'fw3w-apxs', 'xiyt-f6tz',
       'bpt7-i8t8', '58k2-kgtb', '7b32-6xny', 'rxuy-2muj', '2mhq-um7h',
       'cpcf-tcxs', 'pi5s-9p35', '2cd9-59fr', 'ye4j-rp7z', 'nqwf-w8eh',
       'ux7j-iww6', '8ztn-rmii', 's5zg-yzea', 'miz8-534t', 'uvgd-xsc8',
       '27b5-th78', 'szwg-xci6', 'ikvd-dex8', 'j7ww-5ipv', 'a9xv-vek9',
       'm4mp-ji5y', '7acq-q3tq', '6jek-z9ge', 'uhb2-uqfs', 'tpix-uwie',
       '7vu2-3skk', '2jj7-gj6x', 'pask-bcmz', '2de2-6x2h', 'djze-f4qi',
       'yann-8etk', '5dic-xnxs', 'f6st-pb23', 'x9uq-u3qs', 'xgwd-7vhd',
       'exjm-f27b', 'drh3-e2fd', 'gpxw-bq7a', 'urxm-vzzk', 'ajyu-7sgg',
       'vr8g-vfny', 'yusd-j4xi', 'tqmj-j8zm', 'w83z-2kf9', 'khkb-h6hx',
       'v9xd-tt3e', 'i7a5-bsik', 'eizi-ujye', 'pn7c-bqri', 'pv8j-5ywy',
       'h682-ywyg', 'vvdx-b56i', 'e2f7-cs7i', 'yqz9-aduk', 'ch5p-r223',
       'pckb-8r2z', 'ysjj-vb9j', 'ieyi-rqsn', 'arq3-7z49', '72wx-vdjr',
       'asbw-cwm7', 'uh7r-6nya', 'ekax-ky3z', 'qh62-9utz', 'qqsi

Here's the slug format:

`https://nycopendata.socrata.com/api/geospatial/<ENDPOINT ID>?method=export&format=Original`

In [7]:
nyc_geospatial_datasets = []

In [8]:
for dataset in geospatial_datasets:
    endpoint = dataset['resource']['id']
    slug = "https://nycopendata.socrata.com/api/geospatial/" + endpoint + "?method=export&format=Original"
    nyc_geospatial_datasets.append({'endpoint': endpoint, 'resource': slug, 'dataset': '?'})

Ok, but the problem is that our storage format expects us to know the path to the file: `.` if it's a singular file, or to `.shp` if what we've downloaded is a `shapefile` (as is likely). We're going to need to work this out here...

Here is where we hit a problem. Socrata returns data from its API in `chunked` mode without any `content-length` header, so there's *no way* to know ahead of time how much data it's going to send you. In order to avoid certain huge files slowing everything down (there is a 24 GB file hiding amongst links, for example&mdash;some kind of 3D topography of New York City...), we have to "fuzz" a content-length limit using a wall clock time out.

The thing is, "timeout" in the context of the web usually means a connection timeout or inter-packet timeout, both of which `requests` provisions for; it does *not* provision for a wall clock timeout. This was pointed out recently, but is apparently hard to implement and downstreamed to the underlying `urllib3` library; see [here](https://github.com/kennethreitz/requests/issues/3099) and [here](https://github.com/shazow/urllib3/issues/857).

We can work around this by placing our application context into a thread and timing that out instead. Stack Overflow says [you can do this using `eventlet`](http://stackoverflow.com/questions/21965484/timeout-for-python-requests-get-entire-response) in two lines, but the answer given there causes the code to simply hang when I run it in a Jupyter notebook:

`import eventlet
eventlet.monkey_patch()`

The `eventlet` [docs](http://eventlet.net/doc/patching.html) state that it is very important that `eventlet` be imported before as much of everything else as possible, and I think that may be where the problem lies: the Jupyter Python environment is very non-bare even on immediate initialization. I can't how for sure, however. Maybe it's a Windows portability problem for example? Who knows! It just hangs.

The following alternative also hangs:

`
import eventlet
requests = eventlet.import_patched('requests')
with eventlet.Timeout(10):
   requests.get('http://google.com')
`

In fact, this hangs:

`
import eventlet
requests = eventlet.import_patched('requests')
requests.get('http://google.com')
`

What? Ok, at this point I looked at a couple of asychronous `requests` patches, like Reitz's `grequests`. But they didn't include a wall clock feature either...so they didn't solve the problem.

More sleuthing discovers [a ticket on this exact issue](https://github.com/eventlet/eventlet/issues/7), showing that this has been a known and unfixed problem for over three years (!). Here's the suggest workaround:

`
import eventlet
requests = eventlet.import_patched('requests.__init__')
`

But this still hangs too! So `eventlet` is out.

Ok, so the problem is that `requests` can't implement it because it spends all of its download time inside of `urllib3` and can't interrupt that.

It looks like this scenario forces multithreading. We're going to have to implement the download as a `celery` task. This in turn requires that I learn how to use `celery`.

Aside: a possible bottleneck averted, it's possible to get `celery` and `spark` running together: [example](https://github.com/nszceta/spark-python-celery-demo). This is way over my head, however.

Ok so first of all you're going to need a broker and a results backend. The recommended way of doing this is to use `redis` for the backend and `rabbitmq` for the frontend. Find some way of installing `redis` (`choco install redis-64` on Windows, `brew install redis` on Mac, `sudo apt-get install redis-server` on Ubuntu, etc.)  and `rabbitmq` (`choco install rabbitmq` etc.), (maybe not? the latter requires `erlang`...weird requirement...) then launch each process.

I followed [this guide](http://docs.celeryproject.org/en/latest/getting-started/first-steps-with-celery.html#id11). I got stuck almost immediately, on trying to use `add` with results-getting configured. After struggling with it every which way, I Googled around and was shocked to discover that this is [simply not supported at all on Windows](https://github.com/celery/celery/issues/897), with a reporting history going back...4 years.

Oh, and `celery` just [dropped support for Windows altogether](http://stackoverflow.com/questions/37255548/how-to-run-celery-on-windows). `conda` installs a slightly older version of Celery (mid-2016) from before this drop was made.

However, I can't seem to get task time limits to work. Following all of the suggested configurations and running the following:

```python
@app.task(time_limit=1)
def add(x, y):
    import time
    time.sleep(5)
    return x + y
```

...results in it rampantly ignoring the `time_limit` flag.

It's probable that if I tried to get this running on a Mac OSX or on my Ubuntu image it would work out, but no way am I going through that stress for such a simple op.

So! [Multiprocessing](https://docs.python.org/3.5/library/multiprocessing.html) it is.

[This](http://stackoverflow.com/questions/8601856/timing-out-a-multiprocessing-function) answer on Stack Overflow came up first, but I can't seem to implement it for more than one process at a time (`mp.Pool` uses more advanced semantics). That led me to [this](http://stackoverflow.com/questions/37098360/creating-a-timeout-function-in-python-with-multiprocessing?rq=1) answer, which showed a basic implementation of the thing (it's not hard at all, but also not trivial in the underlying hardware; I need to study this library more...) and reccommeded using `pebble` for this.

...ok but Pebble blows up too! It refuses to run, throwing:

`AssertionError: daemonic processes are not allowed to have children`

This has something to do with zombie processes. According to a bug with this message filed in [scikit-learn](https://github.com/automl/auto-sklearn/issues/130#issuecomment-248526406), this may have come about as a result of a change in 3.5. The simple iterator demonstrated in the `pebble` docs works, but using `pd.read_csv` inside of the function does not. Is it because `pd.read_csv` spawns a subprocess? I don't know! ...actually it turned out that I left a decorator out of place. Whoops!